# AA test tutorial 
AA test is important part of randomized controlled experiment, for example AB test. 

The objectives of the AA test are to verify the assumption of uniformity of samples as a result of the applied partitioning method, to select the best partition from the available ones, and to verify the applicability of statistical criteria for checking uniformity. 

For example, there is a hypothesis about the absence of dependence of features on each other. If this hypothesis is not followed, the AA test will fail.

<ul>
  <li><a href="#creation-of-a-new-test-dataset-with-synthetic-data">Creation of a new test dataset with synthetic data.
  <li><a href="#one-split-of-aa-test">One split of AA test.
  <li><a href="#aa-test">AA test.
  <li><a href="#aa-test-with-stratification">AA test with stratification.
</ul>

In [9]:
from hypex import AATest
from hypex.dataset import (
    Dataset,
    InfoRole,
    StratificationRole,
    TargetRole,
    TreatmentRole,
)

## Creation of a new test dataset with synthetic data. 

In order to be able to work with our data in HypEx, first we need to convert it into `dataset`. It is important to mark the data fields by assigning the appropriate `roles`:
- FeatureRole: a role for columns that contain features or predictor variables. Our split will be based on them. Applied by default if the role is not specified for the column.
- TreatmentRole: a role for columns that show the treatment or intervention.
- TargetRole: a role for columns that show the target or outcome variable.
- InfoRole: a role for columns that contain information about the data, such as user IDs. 

In [10]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "gender": StratificationRole(str)
    }, data="data.csv",
)
data

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry
0,0,0,0,488.0,414.444444,NaN,M,E-commerce
1,1,8,1,512.5,462.222222,26.0,NaN,E-commerce
2,2,7,1,483.0,479.444444,25.0,M,Logistics
3,3,0,0,501.5,424.333333,39.0,M,E-commerce
4,4,1,1,543.0,514.555556,18.0,F,E-commerce
...,...,...,...,...,...,...,...,...
9995,9995,10,1,538.5,450.444444,42.0,M,Logistics
9996,9996,0,0,500.5,430.888889,26.0,F,Logistics
9997,9997,3,1,473.0,534.111111,22.0,F,E-commerce
9998,9998,2,1,495.0,523.222222,67.0,F,E-commerce


## AA test
Then we run the experiment on our prepared dataset, wrapped into ExperimentData. In this case we select one of the pre-assembled pipeline, AA_TEST.
We can set the number of iterations for simple execution. In this case the random states are the numbers of each iteration.

In [11]:
aa = AATest(n_iterations=10)
res = aa.execute(data)

In [12]:
res.resume

,feature,group,TTest aa test,KSTest aa test,TTest best split,KSTest best split,result,difference,difference %
0,pre_spends,test,NOT OK,NOT OK,OK,OK,NOT OK,0.173500,0.035626
1,post_spends,test,NOT OK,NOT OK,OK,OK,NOT OK,-0.724844,-0.160177


In [13]:
res.aa_score

,score,pass
pre_spends TTest test,0.653814,False
post_spends TTest test,0.664092,False
pre_spends KSTest test,0.639293,False
post_spends KSTest test,0.637995,False


In [14]:
res.best_split

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry,split
0,0,0,0,488.0,414.444444,NaN,M,E-commerce,control
1,1,8,1,512.5,462.222222,26.0,NaN,E-commerce,test
2,2,7,1,483.0,479.444444,25.0,M,Logistics,test
3,3,0,0,501.5,424.333333,39.0,M,E-commerce,control
4,4,1,1,543.0,514.555556,18.0,F,E-commerce,control
...,...,...,...,...,...,...,...,...,...
9995,9995,10,1,538.5,450.444444,42.0,M,Logistics,control
9996,9996,0,0,500.5,430.888889,26.0,F,Logistics,control
9997,9997,3,1,473.0,534.111111,22.0,F,E-commerce,test
9998,9998,2,1,495.0,523.222222,67.0,F,E-commerce,test


In [15]:
res.best_split_statistic

,feature,group,difference,difference %,TTest pass,TTest p-value,KSTest pass,KSTest p-value
0,pre_spends,test,0.173500,0.035626,OK,0.645746,OK,0.932542
1,post_spends,test,-0.724844,-0.160177,OK,0.357727,OK,0.577046


In [16]:
res.experiments

,splitter_id,pre_spends GroupDifference control mean test,pre_spends GroupDifference test mean test,pre_spends GroupDifference difference test,pre_spends GroupDifference difference % test,post_spends GroupDifference control mean test,post_spends GroupDifference test mean test,post_spends GroupDifference difference test,post_spends GroupDifference difference % test,AASplitter|rs 0| GroupSizes control size test,...,post_spends KSTest pass test,mean TTest p-value,mean TTest pass,mean KSTest p-value,mean KSTest pass,mean test score,post_spends GroupSizes control size test,post_spends GroupSizes test size test,post_spends GroupSizes control size % test,post_spends GroupSizes test size % test
0,AASplitter┴rs 0┴,486.8074,487.3801,0.5727,0.117644,451.724200,452.604911,0.880711,0.194967,5000.0,...,False,0.196474,0.0,0.252129,0.5,0.233577,NaN,NaN,NaN,NaN
1,AASplitter┴rs 1┴,486.8542,487.3333,0.4791,0.098407,452.151400,452.177711,0.026311,0.005819,NaN,...,False,0.588834,0.0,0.490752,0.0,0.523446,5000.0,5000.0,50.0,50.0
2,AASplitter┴rs 2┴,487.1430,487.0445,-0.0985,-0.020220,451.504911,452.824200,1.319289,0.292198,NaN,...,False,0.444120,0.0,0.452796,0.0,0.449904,5000.0,5000.0,50.0,50.0
3,AASplitter┴rs 3┴,487.5133,486.6742,-0.8391,-0.172118,453.078778,451.250333,-1.828444,-0.403560,NaN,...,False,0.023258,1.0,0.130645,0.0,0.094849,5000.0,5000.0,50.0,50.0
4,AASplitter┴rs 4┴,486.9905,487.1970,0.2065,0.042403,451.916489,452.412622,0.496133,0.109784,NaN,...,False,0.556661,0.0,0.362782,0.0,0.427409,5000.0,5000.0,50.0,50.0
5,AASplitter┴rs 5┴,487.2922,486.8953,-0.3969,-0.081450,451.686889,452.642222,0.955333,0.211503,NaN,...,False,0.259216,0.0,0.399740,0.0,0.352899,5000.0,5000.0,50.0,50.0
6,AASplitter┴rs 6┴,486.8775,487.3100,0.4325,0.088831,451.627689,452.701422,1.073733,0.237747,NaN,...,False,0.212447,0.0,0.349031,0.0,0.303503,5000.0,5000.0,50.0,50.0
7,AASplitter┴rs 7┴,487.0070,487.1805,0.1735,0.035626,452.526978,451.802133,-0.724844,-0.160177,NaN,...,False,0.501737,0.0,0.754794,0.0,0.670441,5000.0,5000.0,50.0,50.0
8,AASplitter┴rs 8┴,486.7993,487.3882,0.5889,0.120974,451.924844,452.404267,0.479422,0.106085,NaN,...,False,0.330834,0.0,0.392027,0.5,0.371629,5000.0,5000.0,50.0,50.0
9,AASplitter┴rs 9┴,487.1140,487.0735,-0.0405,-0.008314,452.327511,452.001600,-0.325911,-0.072052,NaN,...,False,0.796888,0.0,0.528860,0.0,0.618203,5000.0,5000.0,50.0,50.0


# AA Test with random states

We can also adjust some of the preset parameters of the experiment by assigning them to the respective params of the experiment. I.e. here we set the range of the random states we want to run our AA test for. 

In [17]:
aa = AATest(random_states=[56, 72, 2, 43])
res = aa.execute(data)

In [18]:
res.resume

,feature,group,TTest aa test,KSTest aa test,TTest best split,KSTest best split,result,difference,difference %
0,pre_spends,test,NOT OK,NOT OK,OK,OK,NOT OK,-0.242900,-0.049855
1,post_spends,test,NOT OK,NOT OK,OK,OK,NOT OK,-0.001956,-0.000432


In [19]:
res.aa_score

,score,pass
pre_spends TTest test,0.652500,False
post_spends TTest test,0.645896,False
pre_spends KSTest test,0.605599,False
post_spends KSTest test,0.533713,False


In [20]:
res.best_split

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry,split
0,0,0,0,488.0,414.444444,NaN,M,E-commerce,control
1,1,8,1,512.5,462.222222,26.0,NaN,E-commerce,test
2,2,7,1,483.0,479.444444,25.0,M,Logistics,control
3,3,0,0,501.5,424.333333,39.0,M,E-commerce,test
4,4,1,1,543.0,514.555556,18.0,F,E-commerce,control
...,...,...,...,...,...,...,...,...,...
9995,9995,10,1,538.5,450.444444,42.0,M,Logistics,test
9996,9996,0,0,500.5,430.888889,26.0,F,Logistics,control
9997,9997,3,1,473.0,534.111111,22.0,F,E-commerce,test
9998,9998,2,1,495.0,523.222222,67.0,F,E-commerce,test


In [21]:
res.best_split_statistic

,feature,group,difference,difference %,TTest pass,TTest p-value,KSTest pass,KSTest p-value
0,pre_spends,test,-0.242900,-0.049855,OK,0.519864,OK,0.694583
1,post_spends,test,-0.001956,-0.000432,OK,0.998020,OK,0.677788


In [22]:
res.experiments

,splitter_id,pre_spends GroupDifference control mean test,pre_spends GroupDifference test mean test,pre_spends GroupDifference difference test,pre_spends GroupDifference difference % test,post_spends GroupDifference control mean test,post_spends GroupDifference test mean test,post_spends GroupDifference difference test,post_spends GroupDifference difference % test,post_spends GroupSizes control size test,...,post_spends TTest pass test,pre_spends KSTest p-value test,pre_spends KSTest pass test,post_spends KSTest p-value test,post_spends KSTest pass test,mean TTest p-value,mean TTest pass,mean KSTest p-value,mean KSTest pass,mean test score
0,AASplitter┴rs 56┴,487.3882,486.7993,-0.5889,-0.120828,451.845400,452.483711,0.638311,0.141268,5000,...,False,0.002465,True,0.744274,False,0.268336,0.0,0.373370,0.5,0.338359
1,AASplitter┴rs 72┴,487.2152,486.9723,-0.2429,-0.049855,452.165533,452.163578,-0.001956,-0.000432,5000,...,False,0.694583,False,0.677788,False,0.758942,0.0,0.686186,0.0,0.710438
2,AASplitter┴rs 2┴,487.1430,487.0445,-0.0985,-0.020220,451.504911,452.824200,1.319289,0.292198,5000,...,False,0.727866,False,0.177727,False,0.444120,0.0,0.452796,0.0,0.449904
3,AASplitter┴rs 43┴,486.8269,487.3606,0.5337,0.109628,452.801000,451.528111,-1.272889,-0.281114,5000,...,False,0.352691,False,0.465358,False,0.131809,0.0,0.409024,0.0,0.316619


# AA Test with stratification

Depending on your requirements it is possible to stratify the data. You can set `stratification=True` and `StratificationRole` in `Dataset` to run it with stratification.  

In [23]:
aa = AATest(random_states=[56, 72, 2, 43], stratification=True)
res = aa.execute(data)

In [24]:
res.resume

,feature,group,TTest aa test,KSTest aa test,TTest best split,KSTest best split,result,difference,difference %
0,pre_spends,test,NOT OK,NOT OK,OK,OK,NOT OK,-0.347111,-0.071236
1,post_spends,test,NOT OK,NOT OK,OK,OK,NOT OK,0.208914,0.046216


In [25]:
res.aa_score

,score,pass
pre_spends TTest test,0.388191,False
post_spends TTest test,0.774438,False
pre_spends KSTest test,0.728226,False
post_spends KSTest test,0.660354,False


In [26]:
res.best_split

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry,split
0,0,0,0,488.0,414.444444,NaN,M,E-commerce,control
1,1,8,1,512.5,462.222222,26.0,NaN,E-commerce,test
2,2,7,1,483.0,479.444444,25.0,M,Logistics,test
3,3,0,0,501.5,424.333333,39.0,M,E-commerce,test
4,4,1,1,543.0,514.555556,18.0,F,E-commerce,control
...,...,...,...,...,...,...,...,...,...
9995,9995,10,1,538.5,450.444444,42.0,M,Logistics,NaN
9996,9996,0,0,500.5,430.888889,26.0,F,Logistics,NaN
9997,9997,3,1,473.0,534.111111,22.0,F,E-commerce,NaN
9998,9998,2,1,495.0,523.222222,67.0,F,E-commerce,NaN


In [27]:
res.best_split_statistic

,feature,group,difference,difference %,TTest pass,TTest p-value,KSTest pass,KSTest p-value
0,pre_spends,test,-0.347111,-0.071236,OK,0.384576,OK,0.129368
1,post_spends,test,0.208914,0.046216,OK,0.801273,OK,0.952433


In [28]:
res.experiments

,splitter_id,pre_spends GroupDifference control mean test,pre_spends GroupDifference test mean test,pre_spends GroupDifference difference test,pre_spends GroupDifference difference % test,post_spends GroupDifference control mean test,post_spends GroupDifference test mean test,post_spends GroupDifference difference test,post_spends GroupDifference difference % test,post_spends GroupSizes control size test,...,post_spends TTest pass test,pre_spends KSTest p-value test,pre_spends KSTest pass test,post_spends KSTest p-value test,post_spends KSTest pass test,mean TTest p-value,mean TTest pass,mean KSTest p-value,mean KSTest pass,mean test score
0,AASplitterWithStratification┴rs 56┴,487.082000,487.110444,0.028444,0.005840,451.633506,452.648938,1.015432,0.224835,4500,...,False,0.612183,False,0.491962,False,0.582179,0.0,0.552072,0.0,0.562108
1,AASplitterWithStratification┴rs 72┴,487.269778,486.922667,-0.347111,-0.071236,452.036765,452.245679,0.208914,0.046216,4500,...,False,0.129368,False,0.952433,False,0.592924,0.0,0.540900,0.0,0.558241
2,AASplitterWithStratification┴rs 2┴,486.928444,487.264000,0.335556,0.068913,452.910272,451.372173,-1.538099,-0.339603,4500,...,False,0.085963,False,0.008355,True,0.232221,0.0,0.047159,0.5,0.108846
3,AASplitterWithStratification┴rs 43┴,487.116556,487.075889,-0.040667,-0.008348,453.141012,451.141432,-1.999580,-0.441271,4500,...,True,0.459584,False,0.105834,False,0.467419,0.5,0.282709,0.0,0.344279


# AA Test by samples 

Depending on your requirements and size of data it is possible to estimate AA test on samples the data. You can set `sample_size=size` to run it. 

In [29]:
aa = AATest(n_iterations=10, sample_size=0.3)
res = aa.execute(data)

In [30]:
res.resume

,feature,group,TTest aa test,KSTest aa test,TTest best split,KSTest best split,result,difference,difference %
0,pre_spends,test,NOT OK,NOT OK,NOT OK,OK,NOT OK,-0.839100,-0.172118
1,post_spends,test,NOT OK,NOT OK,NOT OK,OK,NOT OK,-1.828444,-0.403560


In [31]:
res.aa_score

,score,pass
pre_spends TTest test,0.579749,False
post_spends TTest test,0.596920,False
pre_spends KSTest test,0.502231,False
post_spends KSTest test,0.662001,False


In [32]:
res.best_split

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry,split
0,0,0,0,488.0,414.444444,NaN,M,E-commerce,test
1,1,8,1,512.5,462.222222,26.0,NaN,E-commerce,control
2,2,7,1,483.0,479.444444,25.0,M,Logistics,test
3,3,0,0,501.5,424.333333,39.0,M,E-commerce,test
4,4,1,1,543.0,514.555556,18.0,F,E-commerce,control
...,...,...,...,...,...,...,...,...,...
9995,9995,10,1,538.5,450.444444,42.0,M,Logistics,test
9996,9996,0,0,500.5,430.888889,26.0,F,Logistics,control
9997,9997,3,1,473.0,534.111111,22.0,F,E-commerce,test
9998,9998,2,1,495.0,523.222222,67.0,F,E-commerce,test


In [33]:
res.best_split_statistic

,feature,group,difference,difference %,TTest pass,TTest p-value,KSTest pass,KSTest p-value
0,pre_spends,test,-0.839100,-0.172118,NOT OK,0.026188,OK,0.177727
1,post_spends,test,-1.828444,-0.403560,NOT OK,0.020327,OK,0.083564


In [34]:
res.experiments

,splitter_id,pre_spends GroupDifference control mean test,pre_spends GroupDifference test mean test,pre_spends GroupDifference difference test,pre_spends GroupDifference difference % test,post_spends GroupDifference control mean test,post_spends GroupDifference test mean test,post_spends GroupDifference difference test,post_spends GroupDifference difference % test,post_spends GroupSizes control size test,...,post_spends TTest pass test,pre_spends KSTest p-value test,pre_spends KSTest pass test,post_spends KSTest p-value test,post_spends KSTest pass test,mean TTest p-value,mean TTest pass,mean KSTest p-value,mean KSTest pass,mean test score
0,AASplitter┴rs 0┴,486.707667,487.161882,0.454216,0.093324,452.372963,452.127778,-0.245185,-0.054200,1500,...,False,0.105393,False,0.960105,False,0.607146,0.0,0.532749,0.0,0.557548
1,AASplitter┴rs 1┴,486.513667,487.196118,0.682451,0.140274,451.130889,452.346967,1.216078,0.269562,1500,...,False,0.415281,False,0.146561,False,0.233545,0.0,0.280921,0.0,0.265129
2,AASplitter┴rs 2┴,487.770333,486.974353,-0.795980,-0.163188,452.321852,452.136797,-0.185054,-0.040912,1500,...,False,0.156179,False,0.939361,False,0.499433,0.0,0.547770,0.0,0.531658
3,AASplitter┴rs 3┴,487.175667,487.079294,-0.096373,-0.019782,453.188148,451.983922,-1.204227,-0.265723,1500,...,False,0.973579,False,0.170485,False,0.565251,0.0,0.572032,0.0,0.569772
4,AASplitter┴rs 4┴,487.066333,487.098588,0.032255,0.006622,452.165852,452.164327,-0.001525,-0.000337,1500,...,False,0.989526,False,0.435548,False,0.975117,0.0,0.712537,0.0,0.800064
5,AASplitter┴rs 5┴,487.187667,487.077176,-0.110490,-0.022679,449.811111,452.579869,2.768758,0.615538,1500,...,True,0.936874,False,0.013688,True,0.423253,0.5,0.475281,0.5,0.457938
6,AASplitter┴rs 6┴,486.390333,487.217882,0.827549,0.170141,451.303630,452.316484,1.012854,0.224429,1500,...,False,0.397518,False,0.585789,False,0.238057,0.0,0.491653,0.0,0.407121
7,AASplitter┴rs 7┴,486.354667,487.224176,0.869510,0.178781,453.362889,451.953085,-1.409804,-0.310966,1500,...,False,0.315882,False,0.173681,False,0.150670,0.0,0.244782,0.0,0.213411
8,AASplitter┴rs 8┴,487.042000,487.102882,0.060882,0.012500,452.762963,452.058954,-0.704009,-0.155492,1500,...,False,0.751905,False,0.193861,False,0.715910,0.0,0.472883,0.0,0.553892
9,AASplitter┴rs 9┴,486.539333,487.191588,0.652255,0.134060,453.368296,451.952131,-1.416166,-0.312365,1500,...,False,0.435548,False,0.260911,False,0.208273,0.0,0.348230,0.0,0.301578
